In [241]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [242]:
indexes = pd.read_excel("C:/Users/user/Downloads/data_akbilgic.xlsx")

Начнем с того, что в таблице написаны не сами значения индексов, а их каждодневные относительные изменения, преобразуем значения в относительные относительно значения на момент 4 января 2009 года

In [243]:
columns = list(indexes.iloc[0])
columns[1] = 'ISE_turkish'

Кроме того давайте удалим первый столбец ISE(он изверяется в лирах, у нас уже есть столбец который измеряется в долларах, предсказать курс лиры не наша цель)

In [244]:
from datetime import datetime as dt
indexes.columns = columns
indexes.iloc[0] = ['2009-01-04 00:00:00', '100','100','100','100', '100','100','100','100','100']
indexes = indexes.drop(columns=["ISE_turkish"])
columns = indexes.columns
indexes.date = indexes.date.apply(lambda x: dt.fromisoformat(str(x)))
indexes

,date,ISE,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
0,2009-01-04,100,100,100,100,100,100,100,100
1,2009-01-05,0.038376,-0.004679,0.002193,0.003894,0,0.03119,0.012698,0.028524
2,2009-01-06,0.031813,0.007787,0.008455,0.012866,0.004162,0.01892,0.011341,0.008773
3,2009-01-07,-0.026353,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015
4,2009-01-08,-0.084716,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424
...,...,...,...,...,...,...,...,...,...
532,2011-02-16,0.0134,0.006238,0.001925,0.007952,0.005717,0.018371,0.006975,0.003039
533,2011-02-17,0.015977,0.003071,-0.001186,0.000345,0.00262,0.001686,-0.000581,0.001039
534,2011-02-18,-0.001653,0.001923,0.002872,-0.000723,0.000568,0.005628,0.000572,0.006938
535,2011-02-21,-0.013706,-0.020742,-0.014239,-0.011275,0.001358,-0.011942,-0.012615,-0.000958


In [245]:
def to_float(x):
    try:
        x = float(x)
    except ValueError:
        x = 0
    except TypeError:
        x = 0
    return x
for column in columns[1:]:
    indexes[column] = indexes[column].apply(to_float)

def weekdays(some_date):
    if some_date.weekday() == 5 or some_date.weekday() == 6:
        return None
    return some_date

Будем хранить в таблице относительные значения индексов относительно первого дня(в процентах)

In [246]:
import warnings
warnings.filterwarnings('ignore')
for i in range(1, len(indexes)):
    for column in columns[1:]:
        indexes[column].iloc[i] = indexes[column].iloc[i - 1]*(indexes[column].iloc[i] + 1)
indexes

,date,ISE,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
0,2009-01-04,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,2009-01-05,103.837619,99.532069,100.219342,100.389438,100.000000,103.119023,101.269804,102.852446
2,2009-01-06,107.140978,100.307099,101.066731,101.681009,100.416245,105.069992,102.418270,103.754734
3,2009-01-07,104.317496,97.250828,99.264401,98.759247,102.152736,101.298128,100.669703,101.678040
4,2009-01-08,95.480145,97.580641,98.100399,98.713225,98.060364,104.163159,100.109883,99.703069
...,...,...,...,...,...,...,...,...,...
532,2011-02-16,207.788991,136.006634,140.867201,127.797982,112.691905,157.128192,125.260823,161.510207
533,2011-02-17,211.108878,136.424295,140.700097,127.842077,112.987107,157.393062,125.188107,161.677949
534,2011-02-18,210.759999,136.686626,141.104203,127.749638,113.051315,158.278858,125.259662,162.799651
535,2011-02-21,207.871325,133.851451,139.095040,126.309274,113.204792,156.388744,123.679470,162.643731


Часть EDA уже выполнил мой коллега. Давайте сразу посмотрим насколько точно библиотека prophet предскажет индексы в последние 60 дней, для обучения модели возьмем все дни кроме последних 60

Библиотека prophet от Meta(запрещенная в РФ организация)

In [247]:
import prophet
predictor = prophet.Prophet()

In [248]:
days = 120
predict_days = 181
predictor.fit(df = indexes[['date', 'ISE']][:-days].rename(columns ={'date':'ds','ISE':'y'}))

15:18:44 - cmdstanpy - INFO - Chain [1] start processing


15:18:45 - cmdstanpy - INFO - Chain [1] done processing


In [249]:
future = predictor.make_future_dataframe(periods=predict_days)
forecast = predictor.predict(future)

In [250]:
forecast = forecast.rename(columns = {'ds':'date'})

In [251]:
indexes.dtypes

date       datetime64[ns]
ISE               float64
SP                float64
DAX               float64
FTSE              float64
NIKKEI            float64
BOVESPA           float64
EU                float64
EM                float64
dtype: object

In [258]:
total = pd.merge(forecast, indexes, on = 'date', how = 'inner').dropna()
forecast = total[['date', 'yhat']]
forecast = forecast.rename(columns = {'date':'ds'})

Посчитаем относительную абсолютную ошибку

In [259]:
import numpy as np
# forecast[-days:]
100*np.mean(abs(indexes["ISE"][-days:] - forecast['yhat'][-days:])/indexes["ISE"][-days:])

10.097806422060614

Мы достаточно точно предсказали цену акции, но на самом это не говорит нам о многом, давайте посчитаем как часто мы угадываем вырастет акция в этот день или нет

In [261]:
predict_dynamic = (forecast[-days:].reset_index().yhat - forecast[-days - 1:].reset_index().yhat).drop(index = days)
dynamic = (indexes[-days:].reset_index()['ISE'] - indexes[-days - 1:].reset_index()['ISE']).drop(index = days)
100*np.mean((((dynamic > 0) & (predict_dynamic > 0)) | ((dynamic < 0) & (predict_dynamic < 0))).apply(lambda x: int(x)))

52.5

Не очень точное предсказание, давайте попробуем сгрупировать по неделям

In [265]:
predict_dynamic = (forecast[-days::5].reset_index().yhat - forecast[-days - 5::5].reset_index().yhat).drop(index=24)
dynamic = (indexes[-days::5].reset_index()['ISE'] - indexes[-days - 5::5].reset_index()['ISE']).drop(index=24)
100*np.mean((((dynamic > 0) & (predict_dynamic > 0)) | ((dynamic < 0) & (predict_dynamic < 0))).apply(lambda x: int(x)))

58.333333333333336

Давайте посчитаем и для других колонок

In [268]:
for column in indexes.columns[2:]:
    predictor = prophet.Prophet()
    days = 120
    predict_days = 181
    predictor.fit(df = indexes.reset_index()[['date', column]][:-days].rename(columns ={'date':'ds',column:'y'}))
    future = predictor.make_future_dataframe(periods=predict_days)
    forecast = predictor.predict(future)
    forecast = forecast.rename(columns = {'ds':'date'})
    total = pd.merge(forecast, indexes, on = 'date', how = 'inner').dropna()
    forecast = total[['date', 'yhat']]
    forecast = forecast.rename(columns = {'date':'ds'})
    indexes = indexes.reset_index().drop(columns=['index'])
    last_60_dynamic = (indexes[-days:].reset_index() - indexes[-days - 1:].reset_index()).drop(index = days)[column]
    last_60_dynamic_predict = (forecast.yhat[-days:].reset_index()-forecast.yhat[-days - 1:].reset_index()).drop(index = days)['yhat']
    print(column)
    print(100*np.mean((((last_60_dynamic_predict >= 0) & (last_60_dynamic >= 0)) | ((last_60_dynamic_predict <= 0) & (last_60_dynamic <= 0))).apply(lambda x: int(x))))

15:25:58 - cmdstanpy - INFO - Chain [1] start processing
15:25:58 - cmdstanpy - INFO - Chain [1] done processing
15:25:58 - cmdstanpy - INFO - Chain [1] start processing


SP
46.666666666666664


15:25:59 - cmdstanpy - INFO - Chain [1] done processing
15:25:59 - cmdstanpy - INFO - Chain [1] start processing


DAX
48.333333333333336


15:25:59 - cmdstanpy - INFO - Chain [1] done processing
15:26:00 - cmdstanpy - INFO - Chain [1] start processing


...

BOVESPA
57.49999999999999


15:26:01 - cmdstanpy - INFO - Chain [1] done processing
15:26:01 - cmdstanpy - INFO - Chain [1] start processing


EU
52.5


15:26:01 - cmdstanpy - INFO - Chain [1] done processing


EM
52.5
            ds        yhat
0   2009-01-04  100.125539
1   2009-01-05   94.313044
2   2009-01-06   94.055222
3   2009-01-07   94.348654
4   2009-01-08   94.351458
..         ...         ...
532 2011-02-16  145.511805
533 2011-02-17  145.497581
534 2011-02-18  145.353214
535 2011-02-21  145.457506
536 2011-02-22  145.182656

[537 rows x 2 columns]


Безобразие...Давайте снова попробуем агрегировать по неделям

In [270]:
for column in indexes.columns[2:]:
    predictor = prophet.Prophet()
    days = 120
    predict_days = 181
    predictor.fit(df = indexes.reset_index()[['date', column]][:-days].rename(columns ={'date':'ds',column:'y'}))
    future = predictor.make_future_dataframe(periods=predict_days)
    forecast = predictor.predict(future)
    forecast = forecast.rename(columns = {'ds':'date'})
    total = pd.merge(forecast, indexes, on = 'date', how = 'inner').dropna()
    forecast = total[['date', 'yhat']]
    forecast = forecast.rename(columns = {'date':'ds'})
    indexes = indexes.reset_index().drop(columns=['index'])
    last_60_dynamic = (indexes[-days::5].reset_index() - indexes[-days - 5::5].reset_index())[column]
    last_60_dynamic_predict = (forecast.yhat[-days::5].reset_index()-forecast.yhat[-days - 5::5].reset_index())['yhat']
    print(column)
    print(100*np.mean((((last_60_dynamic_predict >= 0) & (last_60_dynamic >= 0)) | ((last_60_dynamic_predict <= 0) & (last_60_dynamic <= 0))).apply(lambda x: int(x))))

15:27:08 - cmdstanpy - INFO - Chain [1] start processing
15:27:08 - cmdstanpy - INFO - Chain [1] done processing
15:27:08 - cmdstanpy - INFO - Chain [1] start processing


SP
20.0


15:27:08 - cmdstanpy - INFO - Chain [1] done processing
15:27:09 - cmdstanpy - INFO - Chain [1] start processing


DAX
64.0


15:27:09 - cmdstanpy - INFO - Chain [1] done processing
15:27:10 - cmdstanpy - INFO - Chain [1] start processing


...

BOVESPA
44.0


15:27:11 - cmdstanpy - INFO - Chain [1] done processing
15:27:11 - cmdstanpy - INFO - Chain [1] start processing


EU
32.0


15:27:11 - cmdstanpy - INFO - Chain [1] done processing


EM
36.0


Тем не менее давайте посмотрим на конечное значение индексов

In [274]:
for column in indexes.columns[1:]:
    predictor = prophet.Prophet()
    days = 120
    predict_days = 181
    predictor.fit(df = indexes.reset_index()[['date', column]][:-days].rename(columns ={'date':'ds',column:'y'}))
    future = predictor.make_future_dataframe(periods=predict_days)
    forecast = predictor.predict(future)
    forecast = forecast.rename(columns = {'ds':'date'})
    total = pd.merge(forecast, indexes, on = 'date', how = 'inner').dropna()
    forecast = total[['date', 'yhat']]
    forecast = forecast.rename(columns = {'date':'ds'})
    print(column)
    print(f'мы предсказали {forecast.yhat.iloc[536]} на самом деле: {indexes[column].iloc[536]}')

15:32:42 - cmdstanpy - INFO - Chain [1] start processing
15:32:42 - cmdstanpy - INFO - Chain [1] done processing


ISE
мы предсказали 204.51648404910048 на самом деле: 203.8299222461267


15:32:43 - cmdstanpy - INFO - Chain [1] start processing
15:32:43 - cmdstanpy - INFO - Chain [1] done processing
15:32:43 - cmdstanpy - INFO - Chain [1] start processing


SP
мы предсказали 93.9163235192495 на самом деле: 133.85145128890417


15:32:43 - cmdstanpy - INFO - Chain [1] done processing


...

BOVESPA
мы предсказали 138.29631611935667 на самом деле: 154.47273839507542


15:32:46 - cmdstanpy - INFO - Chain [1] done processing
15:32:46 - cmdstanpy - INFO - Chain [1] start processing


EU
мы предсказали 96.25270648148243 на самом деле: 123.00357465499603


15:32:46 - cmdstanpy - INFO - Chain [1] done processing


EM
мы предсказали 145.18265627213026 на самом деле: 160.31842522457436


Наиболее удачно получилось построить 'ISE', давайте построим для него график

In [275]:
def weekdays(some_date):
    if some_date.weekday() == 5 or some_date.weekday() == 6:
        return None
    return some_date
import plotly.graph_objects as go
predictor = prophet.Prophet()
days = 120
predict_days = 181
indexes[['date', "ISE"]][:-days].rename(columns ={'date':'ds','ISE':'y'})
predictor.fit(df = indexes.reset_index()[['date', 'ISE']][:-days].rename(columns ={'date':'ds','ISE':'y'}))
future = predictor.make_future_dataframe(periods=predict_days)
forecast = predictor.predict(future)
forecast = forecast.rename(columns = {'ds':'date'})
total = pd.merge(forecast, indexes, on = 'date', how = 'inner').dropna()
forecast = total[['date', 'yhat']]
forecast = forecast.rename(columns = {'date':'ds'})
fig = go.Figure()
fig.add_trace(go.Scatter(x = indexes['date'][-days:], y = indexes['ISE'][-days:]))
fig.add_trace(go.Scatter(x = forecast['ds'][-days:], y = forecast['yhat'][-days:]))
fig.show()
print(forecast.iloc[-1])
print(indexes.iloc[-1])

15:33:31 - cmdstanpy - INFO - Chain [1] start processing
15:33:31 - cmdstanpy - INFO - Chain [1] done processing


ds      2011-02-22 00:00:00
yhat             204.516484
Name: 536, dtype: object
date       2011-02-22 00:00:00
ISE                 203.829922
SP                  133.851451
DAX                 139.029294
FTSE                125.930712
NIKKEI              111.176204
BOVESPA             154.472738
EU                  123.003575
EM                  160.318425
Name: 536, dtype: object


В целом можно сделать вывод, что prophet больше подходит для больших датасетов и больших горизонтов предсказания, поэтому и предсказание на этом датасете такое неточное